In [1]:
import tensorflow as tf

In [6]:
def get_classes(folder):
    return [x[0].split('/')[-1] for x in os.walk(folder)][1:]


def read_and_flatten(class_dir, ground_truth):
    fnames = [(os.path.join(class_dir, f), ground_truth) 
              for f in os.listdir(class_dir) 
              if os.path.isfile(os.path.join(class_dir, f))]
    return fnames

def preprocess(directory):
    """
    Returns: dict of tuples: (filename: ground truth index)
    """
    classes = get_classes(directory)
    num_classes = len(classes)
    
    imgs = {}
    for i in range(num_classes):
        imgs.update(read_and_flatten(directory +'/'+ classes[i], i))
    return imgs

def all_filenames(label_dir):
    with tf.Graph() as graph:
        with graph.name_scope("filename_aggregation") as graph.scope:
            
            # Make a list of all image filepaths in this label_dir
            filenames = tf.train.match_filenames_once(label_dir, name="filename_aggregation")
    return graph

base_dir = '/Users/anjueappen/png/arm'
with tf.Session(graph=all_filenames(base_dir)) as sess:
    tf.initialize_all_variables()
    result = sess.run(sess.graph.get_tensor_by_name("filename_aggregation:0"))
    print(result)


AttributeError: __exit__

In [ ]:
from sklearn.model_selection import KFold

class ImgBatchMaker():
    
    def __init__(self, base_dir, num_splits):
        self.base_dir = base_dir
        
    
    def get_classes(folder):
        return [x[0].split('/')[-1] for x in os.walk(folder)][1:]


    def read_and_flatten(class_dir, ground_truth):
        fnames = [(os.path.join(class_dir, f), ground_truth) 
                  for f in os.listdir(class_dir) 
                  if os.path.isfile(os.path.join(class_dir, f))]
        return fnames

    def filename_mapping(self):
        """
        Returns: dict of tuples: (filename: ground truth index)
        """
        classes = get_classes(self.base_dir)
        num_classes = len(classes)

        imgs = {}
        for i in range(num_classes):
            imgs.update(read_and_flatten(directory +'/'+ classes[i], i))
        return imgs

    def split():

    def img_decode()
    

In [ ]:
class TFBatchMaker():
    
    def __init__():
        

    def getExamplesGenerator():
        def examplesGenerator(q):
            while True:
                q.put(getExample())

        queue = multiprocessing.Queue(maxsize=128)
        for w in range(0, 16):
            proc = multiprocessing.Process(target=examplesGenerator, args=(queue,))
            proc.start()

        while True:
            yield queue.get()

"""
1. Start with dict of filepaths and truth indexes 
2. Split with KFold: 18000 train images, 2000 test images  
3. Generate batches for 18000 trains -> tf.batch()
4. Create input and ground truth tensors for each batch 
"""

In [3]:
# Placeholder for total number of labels (250 in our case)
num_labels = tf.placeholder(tf.int32)

# Placeholder for dirpath for a specific label (i.e. /user/local/png/airplane)
label_dir = tf.placeholder(tf.string) 

# Placeholder for index that we want to set in ground truth array for all images
# in the directory label_dir 
index = tr.placeholder(tf.int32)

# Make a queue of all image filepaths in label_dir
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(label_dir))

# Initialize the file reader that we will use for filename_queue
image_reader = tf.WholeFileReader()

# Read a whole file from the queue, the first returned value in the tuple is the
# filename which we are ignoring.
_, image_file = image_reader.read(filename_queue)

# Decode the image as a PNG file, this will turn it into a Tensor which we can
# then use in training.
image = tf.image.decode_png(image_file)


# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
    tf.initialize_all_variables().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Get an image tensor and print its value.
    image_tensor = sess.run([image])
    print(image_tensor)

    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


NotFoundError: ~/png
	 [[Node: matching_filenames/MatchingFiles = MatchingFiles[_device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames/MatchingFiles/pattern)]]

Caused by op 'matching_filenames/MatchingFiles', defined at:
  File "/Users/anjueappen/miniconda2/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/anjueappen/miniconda2/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/anjueappen/miniconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/anjueappen/miniconda2/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/anjueappen/miniconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/anjueapp

In [ ]:
"""
1. Start with dict of filepaths and truth indexes 
2. Split with KFold: 18000 train images, 2000 test images  
3. Generate batches for 18000 trains -> tf.batch()
4. Create input and ground truth tensors for each batch 
"""

